In [26]:
import os
import pandas as pd
from tqdm import tqdm
import sys

import warnings
warnings.simplefilter("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [27]:
program_id = "species_net_bounded"
num_epochs = 15

In [28]:
# get image filepaths
pangolin_path = "[6-22] Bounded Pangolin Images"
other_path = "[6-22] Bounded Other Animal Images"

pangolin_images = os.listdir(pangolin_path)
other_images = os.listdir(other_path)
print(len(pangolin_images), pangolin_images)
print(len(other_images), other_images)

206 ['IMG_0006.JPG', 'IMG_0015.JPG', 'IMG_0016.JPG', 'IMG_0017.JPG', 'IMG_0018.JPG', 'IMG_0153.JPG', 'IMG_0155.JPG', 'IMG_0156.JPG', 'IMG_0157.JPG', 'IMG_0158.JPG', 'IMG_0159.JPG', 'IMG_0160.JPG', 'IMG_0161.JPG', 'IMG_0162.JPG', 'IMG_0179.JPG', 'IMG_0180.JPG', 'IMG_0181.JPG', 'IMG_0182.JPG', 'IMG_0236.JPG', 'IMG_0247.JPG', 'IMG_0251.JPG', 'IMG_0252.JPG', 'IMG_0253.JPG', 'IMG_0254.JPG', 'IMG_0255.JPG', 'IMG_0256.JPG', 'IMG_0257.JPG', 'IMG_0258.JPG', 'IMG_0259.JPG', 'IMG_0271.JPG', 'IMG_0416.JPG', 'IMG_0417.JPG', 'IMG_0418.JPG', 'IMG_0420.JPG', 'IMG_0422.JPG', 'IMG_0423.JPG', 'IMG_0424.JPG', 'IMG_0425.JPG', 'IMG_0426.JPG', 'IMG_0549.JPG', 'IMG_0550.JPG', 'IMG_0551.JPG', 'IMG_0552.JPG', 'IMG_0553.JPG', 'IMG_0554.JPG', 'IMG_0555.JPG', 'IMG_0556.JPG', 'IMG_0557.JPG', 'IMG_0558.JPG', 'IMG_0559.JPG', 'IMG_0560.JPG', 'IMG_0561.JPG', 'IMG_0562.JPG', 'IMG_0563.JPG', 'IMG_0564.JPG', 'IMG_0577.JPG', 'IMG_0578.JPG', 'IMG_0579.JPG', 'IMG_0581.JPG', 'IMG_0582.JPG', 'IMG_0583.JPG', 'IMG_0584.JPG', 'IM

In [29]:
# combine filepath and label
data = ([(os.path.join(pangolin_path, img), 1) for img in pangolin_images if img != ".DS_Store"] +
    [(os.path.join(other_path, img), 0) for img in other_images])

print(data)
print(len(data))

[('[6-22] Bounded Pangolin Images\\IMG_0006.JPG', 1), ('[6-22] Bounded Pangolin Images\\IMG_0015.JPG', 1), ('[6-22] Bounded Pangolin Images\\IMG_0016.JPG', 1), ('[6-22] Bounded Pangolin Images\\IMG_0017.JPG', 1), ('[6-22] Bounded Pangolin Images\\IMG_0018.JPG', 1), ('[6-22] Bounded Pangolin Images\\IMG_0153.JPG', 1), ('[6-22] Bounded Pangolin Images\\IMG_0155.JPG', 1), ('[6-22] Bounded Pangolin Images\\IMG_0156.JPG', 1), ('[6-22] Bounded Pangolin Images\\IMG_0157.JPG', 1), ('[6-22] Bounded Pangolin Images\\IMG_0158.JPG', 1), ('[6-22] Bounded Pangolin Images\\IMG_0159.JPG', 1), ('[6-22] Bounded Pangolin Images\\IMG_0160.JPG', 1), ('[6-22] Bounded Pangolin Images\\IMG_0161.JPG', 1), ('[6-22] Bounded Pangolin Images\\IMG_0162.JPG', 1), ('[6-22] Bounded Pangolin Images\\IMG_0179.JPG', 1), ('[6-22] Bounded Pangolin Images\\IMG_0180.JPG', 1), ('[6-22] Bounded Pangolin Images\\IMG_0181.JPG', 1), ('[6-22] Bounded Pangolin Images\\IMG_0182.JPG', 1), ('[6-22] Bounded Pangolin Images\\IMG_0236.JP

In [30]:
# create and write column headers to csv files for logging purposes
import csv
log_folder_path = "./log/" + program_id + "/"
os.makedirs(log_folder_path, exist_ok=True)

train_csv_file_path = log_folder_path + "train_scores.csv"
val_csv_file_path = log_folder_path + "val_scores.csv"
time_csv_file_path = log_folder_path + "time_per_epoch.csv"
test_csv_file_path = log_folder_path + "test_scores.csv"
loss_csv_file_path = log_folder_path + "loss_per_epoch.csv"

train_csv = open(train_csv_file_path, mode='w', newline='')
val_csv = open(val_csv_file_path, mode='w', newline='')
time_csv = open(time_csv_file_path, mode='w', newline='')
test_csv = open(test_csv_file_path, mode='w', newline='')
loss_csv = open(loss_csv_file_path, mode='w', newline='')

train_writer = csv.writer(train_csv)
val_writer = csv.writer(val_csv)
time_writer = csv.writer(time_csv)
test_writer = csv.writer(test_csv)
loss_writer = csv.writer(loss_csv)

train_writer.writerow([
    'fold', 'epoch',
    'accuracy_pangolin', 'accuracy_other',
    'recall_pangolin', 'recall_other',
    'precision_pangolin', 'precision_other',
    'f1_pangolin', 'f1_other',
    'auc_pangolin', 'auc_other'
])
val_writer.writerow([
    'fold', 'epoch',
    'accuracy_pangolin', 'accuracy_other',
    'recall_pangolin', 'recall_other',
    'precision_pangolin', 'precision_other',
    'f1_pangolin', 'f1_other',
    'auc_pangolin', 'auc_other'
])
time_writer.writerow(['fold', 'epoch', 'time'])
test_writer.writerow([
    'fold',
    'accuracy_pangolin', 'accuracy_other',
    'recall_pangolin', 'recall_other',
    'precision_pangolin', 'precision_other',
    'f1_pangolin', 'f1_other',
    'auc_pangolin', 'auc_other'
])
loss_writer.writerow(['fold', 'epoch', 'train_loss', 'val_loss'])

32

In [31]:
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import torchvision.transforms.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
)

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# plot image of misclassifications
def plot_misclassified(fold, pred, data):
    true = [label for _, label in data]
    fn_indices = [i for i, (p, t) in enumerate(zip(pred, true)) if p == 0 and t == 1]   # false negatives
    fp_indices = [i for i, (p, t) in enumerate(zip(pred, true)) if p == 1 and t == 0]   # false positives

    # helper method to plot image
    def plot_images(indices, title):
        if not indices:
            print(f"No Misclassified Samples Found: " + title)
            return
        
        print(f"Misclassified Samples Found: " + str(len(indices)))
        indices = indices[:5]
        
        fig, axes = plt.subplots(1, len(indices), figsize=(15, 5))
        if len(indices) == 1:
            axes = [axes]
        
        for ax, idx in zip(axes, indices):
            filename, true_label = data[idx]
            img = Image.open(filename)
            
            ax.imshow(img)
            ax.axis("off")
            ax.set_title(f"{title}\nPred: {pred[idx]}, True: {true_label}", fontsize=10)
        plt.show()

    # plot images
    plot_images(fn_indices, "Fold " + str(fold + 1) + "- False Negatives")
    plot_images(fp_indices, "Fold " + str(fold + 1) + "- False Positives")

###############################################################################################################################################
# printing/logging methods (different for validation, training, and testing because they have different ways to log to csv files)
###############################################################################################################################################

# prints score for validation
def val_print_and_log_scores(fold, epoch, true, pred, prob):
    # calculating scores for pangolin class
    accuracy = accuracy_score(true, pred)
    pangolin_precision = precision_score(true, pred, pos_label=1, zero_division=0)
    pangolin_recall = recall_score(true, pred, pos_label=1, zero_division=0)
    pangolin_f1 = f1_score(true, pred, pos_label=1, zero_division=0)
    pangolin_prob = prob
    pangolin_auc = roc_auc_score(true, pangolin_prob)

    print(f"\tpangolin accuracy: {accuracy:.4f}")
    print(f"\tpangolin precision: {pangolin_precision:.4f}")
    print(f"\tpangolin recall: {pangolin_recall:.4f}")
    print(f"\tpangolin f1-score: {pangolin_f1:.4f}")
    print(f"\tpangolin auc: {pangolin_auc:.4f}")

    # calculating scores for other class
    other_precision = precision_score(true, pred, pos_label=0, zero_division=0)
    other_recall = recall_score(true, pred, pos_label=0, zero_division=0)
    other_f1 = f1_score(true, pred, pos_label=0, zero_division=0)

    other_prob = prob
    true_inverted = [1 if t != 1 else 0 for t in true]
    other_auc = roc_auc_score(true_inverted, other_prob)

    print(f"\tother accuracy: {accuracy:.4f}")
    print(f"\tother precision: {other_precision:.4f}")
    print(f"\tother recall: {other_recall:.4f}")
    print(f"\tother f1-score: {other_f1:.4f}")
    print(f"\tother auc: {other_auc:.4f}")

    # writing to csv file
    val_writer.writerow([
            fold, epoch,
            accuracy, accuracy,
            pangolin_recall, other_recall,
            pangolin_precision, other_precision,
            pangolin_f1, other_f1,
            pangolin_auc, other_auc
    ])


# print and log score for training
def train_print_and_log_scores(fold, epoch, true, pred, prob):
    # calculating scores for pangolin class
    accuracy = accuracy_score(true, pred)
    pangolin_precision = precision_score(true, pred, pos_label=1, zero_division=0)
    pangolin_recall = recall_score(true, pred, pos_label=1, zero_division=0)
    pangolin_f1 = f1_score(true, pred, pos_label=1, zero_division=0)
    pangolin_prob = prob.detach().numpy()
    pangolin_auc = roc_auc_score(true, pangolin_prob)

    print(f"\tpangolin accuracy: {accuracy:.4f}")
    print(f"\tpangolin precision: {pangolin_precision:.4f}")
    print(f"\tpangolin recall: {pangolin_recall:.4f}")
    print(f"\tpangolin f1-score: {pangolin_f1:.4f}")
    print(f"\tpangolin auc: {pangolin_auc:.4f}")

    # calculating scores for other class
    other_precision = precision_score(true, pred, pos_label=0, zero_division=0)
    other_recall = recall_score(true, pred, pos_label=0, zero_division=0)
    other_f1 = f1_score(true, pred, pos_label=0, zero_division=0)
    other_prob = prob.detach().numpy()
    true_inverted = (true != 1).detach().numpy().astype(int)  
    other_auc = roc_auc_score(true_inverted, other_prob)

    print(f"\tother accuracy: {accuracy:.4f}")
    print(f"\tother precision: {other_precision:.4f}")
    print(f"\tother recall: {other_recall:.4f}")
    print(f"\tother f1-score: {other_f1:.4f}")
    print(f"\tother auc: {other_auc:.4f}")

    # writing to csv file
    train_writer.writerow([
        fold, epoch,
        accuracy, accuracy,
        pangolin_recall, other_recall,
        pangolin_precision, other_precision,
        pangolin_f1, other_f1,
        pangolin_auc, other_auc
    ])

# print and log scores for testing
def test_print_and_log_scores(fold, true, pred, prob):
    # calculating scores for pangolin class
    accuracy = accuracy_score(true, pred)
    pangolin_precision = precision_score(true, pred, pos_label=1, zero_division=0)
    pangolin_recall = recall_score(true, pred, pos_label=1, zero_division=0)
    pangolin_f1 = f1_score(true, pred, pos_label=1, zero_division=0)
    pangolin_prob = prob
    pangolin_auc = roc_auc_score(true, pangolin_prob)

    print(f"\tpangolin accuracy: {accuracy:.4f}")
    print(f"\tpangolin precision: {pangolin_precision:.4f}")
    print(f"\tpangolin recall: {pangolin_recall:.4f}")
    print(f"\tpangolin f1-score: {pangolin_f1:.4f}")
    print(f"\tpangolin auc: {pangolin_auc:.4f}")

    # calculating scores for other class
    other_precision = precision_score(true, pred, pos_label=0, zero_division=0)
    other_recall = recall_score(true, pred, pos_label=0, zero_division=0)
    other_f1 = f1_score(true, pred, pos_label=0, zero_division=0)
    other_prob = prob
    true_inverted = [1 if t != 1 else 0 for t in true]
    other_auc = roc_auc_score(true_inverted, other_prob)

    print(f"\tother accuracy: {accuracy:.4f}")
    print(f"\tother precision: {other_precision:.4f}")
    print(f"\tother recall: {other_recall:.4f}")
    print(f"\tother f1-score: {other_f1:.4f}")
    print(f"\tother auc: {other_auc:.4f}")

    # writing to csv file
    test_writer.writerow([
        fold,
        accuracy, accuracy,
        pangolin_recall, other_recall,
        pangolin_precision, other_precision,
        pangolin_f1, other_f1,
        pangolin_auc, other_auc
    ])

# custom collate function for the data loader
def collate_fn(batch):
    images = torch.stack([item["image"] for item in batch])
    labels = torch.tensor([item["label"] for item in batch], dtype=torch.long)
    image_ids = [item["image_id"] for item in batch]

    return {"image": images, "label": labels, "image_id": image_ids}

def crop_bottom_bar(image):
    width, height = image.size
    if (width != 5376 and height != 3024): return image
    bar_height = int(height * 0.05)
    cropped_image = F.crop(image, 0, 0, height - bar_height, width)
    return cropped_image

# custom image dataset
class ImagesDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.transform = transforms.Compose(
            [
                transforms.Lambda(crop_bottom_bar),
                transforms.Grayscale(num_output_channels=3),
                transforms.Resize((480, 480)),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
                )
            ]
        )
        
    def __getitem__(self, index):
        filepath, label = self.data[index]
        image = Image.open(filepath).convert("RGB")
        image = self.transform(image)
        label = torch.tensor(label, dtype=torch.long)
        sample = {"image": image, "label": label, "image_id": index}
        return sample

    def __len__(self):
        return len(self.data)

# store feature maps for each layer
feature_maps = {}

# hook function to save feature maps
def hook_fn(module, input, output, name):
    feature_maps[name] = output.detach()

# layers_to_visualize = [
#     'fx_model.SpeciesNet/efficientnetv2-m/stem_conv/Conv2D.1',               # very early
#     'fx_model.SpeciesNet/efficientnetv2-m/block2a_expand_conv/Conv2D',       # early-mid
#     'fx_model.SpeciesNet/efficientnetv2-m/block4a_project_conv/Conv2D',      # mid
#     'fx_model.SpeciesNet/efficientnetv2-m/block6b_project_conv/Conv2D',      # late-mid
#     'fx_model.SpeciesNet/efficientnetv2-m/top_conv/Conv2D'                   # final conv
# ]

layers_to_visualize = [
    'SpeciesNet/efficientnetv2-m/stem_conv/Conv2D.1',               # very early
    'SpeciesNet/efficientnetv2-m/block2a_expand_conv/Conv2D',       # early-mid
    'SpeciesNet/efficientnetv2-m/block4a_project_conv/Conv2D',      # mid
    'SpeciesNet/efficientnetv2-m/block6b_project_conv/Conv2D',      # late-mid
    'SpeciesNet/efficientnetv2-m/top_conv/Conv2D'                   # final conv
]



In [32]:
import torch.optim as optim
from torch import nn
from speciesnet import SpeciesNet
from sklearn.metrics import ConfusionMatrixDisplay
import time

pretrained_model = "./pretrained_model.pth"       # filepath to pretrained model check point
speciestnet_model_path = './speciesnet_model'

model_folder_path = "./models/" + program_id + "/"                 # create folder to store optimal model per fold
os.makedirs(model_folder_path, exist_ok=True)

In [33]:
# read cross validation indices from stored csv file
df_splits = pd.read_csv("crossval_splits.csv")
print(df_splits)

      fold  index   type
0        0    375  train
1        0    112  train
2        0     35  train
3        0    221  train
4        0    426  train
...    ...    ...    ...
3260     4    634   test
3261     4    636   test
3262     4    641   test
3263     4    645   test
3264     4    648   test

[3265 rows x 3 columns]


In [34]:
class WrappedModel(nn.Module):
    def __init__(self, fx_model):
        super().__init__()
        self.fx_model = fx_model

    def forward(self, x):
        # Ensure input is tracked
        x = x.clone().detach().requires_grad_(True)
        out = self.fx_model(x)
        # Reattach to computation graph
        if not out.requires_grad:
            out = out * 1.0
        return out

In [35]:
pangolin_indices = []
with open("./speciesnet_model/always_crop_99710272_22x8_v12_epoch_00148.labels.txt", "r") as f:
    for idx, line in enumerate(f):
        if "pangolin" in line.lower():
            pangolin_indices.append(idx)
print(pangolin_indices)


[70, 881, 951, 1044, 1404, 1405, 2330]


In [36]:
# lines 882, 1405, 1406
# 2f336cdf-a62f-4587-a516-6e6c74d07353;mammalia;pholidota;manidae;phataginus;tricuspis;white-bellied pangolin
# b1cefdc9-af34-4f28-b077-1186dd6b5072;mammalia;pholidota;manidae;;;pangolin family
# ade3ecab-c110-429a-849e-b6afdb290219;mammalia;pholidota;manidae;manis;;pangolin species
# pangolin_indices = [881, 1404, 1405]


In [37]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [133]:
for fold in df_splits["fold"].unique():    
    # get the stored indices 
    train_index = df_splits[(df_splits["fold"] == fold) & (df_splits["type"] == "train")]["index"].values
    val_index = df_splits[(df_splits["fold"] == fold) & (df_splits["type"] == "val")]["index"].values
    test_index = df_splits[(df_splits["fold"] == fold) & (df_splits["type"] == "test")]["index"].values

    print(f"\n\nfold {fold + 1} -------------------------------------------------------------------------------------------------")

    min_val_loss = sys.float_info.max
    model_name = "model_fold" + str(fold + 1) + ".pth"
    model_path = model_folder_path + "/" + model_name       # path to store the model checkpoints, name-specific to fold

    # split data and get dataloaders
    train_data = [data[i] for i in train_index]
    val_data = [data[i] for i in val_index]
    test_data = [data[i] for i in test_index]

    train_dataset = ImagesDataset(train_data)
    val_dataset = ImagesDataset(val_data)
    test_dataset = ImagesDataset(test_data)

    train_dataloader = DataLoader(train_dataset, batch_size=32, collate_fn=collate_fn)
    val_dataloader = DataLoader(val_dataset, batch_size=32, collate_fn=collate_fn)
    test_dataloader = DataLoader(test_dataset, batch_size=32, collate_fn=collate_fn)

    # load model
    speciesnet_model = SpeciesNet(speciestnet_model_path)
    base_model = speciesnet_model.classifier.model
    # base_model = WrappedModel(base_model)
    base_model.fc = nn.Sequential(
        nn.Linear(2048, 100),
        nn.ReLU(inplace=True),
        nn.Dropout(0.1),
        nn.Linear(100, 1),  # output a single value for binary classification
        # nn.Sigmoid()
    )

    model = base_model.to(device)
    
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

    for name, param in model.named_parameters():
        if 'block7' in name or 'top_conv' in name or 'fc' in name:
            param.requires_grad = True
        else:
            param.requires_grad = False

    # register hooks for each layer
    hooks = []
    for layer_name in layers_to_visualize:
        layer = dict([*model.named_modules()])[layer_name]  # Get the layer by name
        hook = layer.register_forward_hook(lambda module, input, output, name=layer_name: hook_fn(module, input, output, name))
        hooks.append(hook)

    for epoch in range(1, num_epochs + 1):
        start_time = time.time()

        print(f"\nepoch {epoch}")

        #############################################################################################################################################
        # training
        #############################################################################################################################################
        print("TRAINING")

        model.train()

        tracking_loss = {}
        training_loss = 0
        training_num_loss = 0

        all_outputs = []
        all_labels = []

        # iterate through the dataloader batches. tqdm keeps track of progress.
        for batch_n, batch in tqdm(
            enumerate(train_dataloader), total=len(train_dataloader)
        ):
            
            feature_map_images = batch['image']            

            batch["image"] = batch["image"].permute(0, 2, 3, 1)     
    
            images = batch['image'].to(device)
            labels = batch['label'].float().to(device)

            image_ids = batch['image_id']

            # 1) zero out the parameter gradients so that gradients from previous batches are not used in this step
            optimizer.zero_grad()

            # 2) run the foward step on this batch of images
            outputs = model(images).squeeze()

            pangolin_logits = outputs[:, pangolin_indices].logsumexp(dim=1)
            pangolin_prob = torch.sigmoid(pangolin_logits)

            # 3) compute the loss
            loss = criterion(pangolin_logits, labels)

            training_loss += loss.item()
            training_num_loss += 1

            # let's keep track of the loss by epoch
            tracking_loss[epoch] = loss.item()

            # 4) compute our gradients
            # for name, param in model.named_parameters():
            #     if torch.isnan(param).any():
                    # print(f"NaN in weights: {name}")

            loss.backward()

            # for name, param in model.named_parameters():
            #     if torch.isnan(param).any():
            #         print(f"NaN in weights: {name}")

            # update our weights
            optimizer.step()

            all_outputs.append(pangolin_prob.detach().cpu())
            all_labels.append(labels.detach().cpu())

        all_outputs = torch.cat(all_outputs, dim=0)
        all_labels = torch.cat(all_labels, dim=0)

        train_pred = (all_outputs > 0.5).float()     # get the predicted labels
        train_prob = all_outputs                    # get the probability of the positive and negative class
        train_true = all_labels             # get the true labels

        # print and log scores
        training_loss /= training_num_loss
        print(f"\ttraining loss: {training_loss:.4f}")
        train_print_and_log_scores(fold, epoch, train_true, train_pred, train_prob)

        # outputting images for feature extraction
        # num_images = 10
        # random_indices = np.random.choice(len(feature_map_images), num_images, replace=False)

        # mean = [0.485, 0.456, 0.406]
        # std = [0.229, 0.224, 0.225]

        # label_1_indices = [i for i in range(len(image_ids)) if train_data[image_ids[i]][1] == 1]
        # label_0_indices = [i for i in range(len(image_ids)) if train_data[image_ids[i]][1] == 0]

        # sampled_indices = random.sample(label_1_indices, 3) + random.sample(label_0_indices, 7)

        # for img_idx in sampled_indices:
        #     file_name, label = train_data[image_ids[img_idx]]
        #     fig, axes = plt.subplots(1, len(layers_to_visualize) + 2, figsize=(30, 5))
        #     fig.suptitle(f"Label: {label}", fontsize=20)

        #     image = (feature_map_images[img_idx].cpu().numpy().transpose(1, 2, 0)) * std + mean
        #     image = np.clip(image, 0, 1)

        #     # image
        #     axes[0].imshow(image)
        #     axes[0].set_title("original")
        #     axes[0].axis("off")

        #     for i, layer_name in enumerate(layers_to_visualize):
        #         fmap = feature_maps[layer_name][img_idx]
        #         fmap = fmap.mean(dim=0)
        #         fmap = (fmap - fmap.min()) / (fmap.max() - fmap.min())

        #         if fmap.ndimension() == 3:
        #             num_channels = fmap.shape[0]
        #             random_channel = np.random.randint(num_channels)
        #             fmap_to_show = fmap[random_channel].cpu().numpy()
        #         else:
        #             random_channel = 0
        #             fmap_to_show = fmap.cpu().numpy()

        #         fmap_to_show = (fmap_to_show - fmap_to_show.min()) / (fmap_to_show.max() - fmap_to_show.min() + 1e-5)
        #         axes[i + 1].imshow(fmap_to_show, cmap='viridis')
        #         axes[i + 1].set_title(f"{layer_name} feature map {random_channel}")
        #         axes[i + 1].axis("off")

        #     plt.show()

        #############################################################################################################################################
        # validation
        #############################################################################################################################################
        print("VALIDATION")

        val_preds_collector = []
        model.eval()

        val_loss = 0.0
        val_num_loss = 0

        # iterate through dataloader and run the model
        with torch.no_grad():
            for batch in tqdm(val_dataloader, total=len(val_dataloader)):
                batch["image"] = batch["image"].permute(0, 2, 3, 1)     
        
                images = batch['image'].to(device)
                labels = batch['label'].float().to(device)

                image_ids = batch['image_id']
                logits = model.forward(images)
                
                pangolin_logits = logits[:, pangolin_indices].logsumexp(dim=1)
                pangolin_prob = torch.sigmoid(pangolin_logits)

                loss = criterion(pangolin_logits, labels)

                val_loss += loss.item()
                val_num_loss += 1

                preds_df = pd.DataFrame(
                    pangolin_prob.detach().cpu().numpy(),
                    index=batch["image_id"],
                    columns=["prob"]
                )
                val_preds_collector.append(preds_df)


        val_preds_df = pd.concat(val_preds_collector)
        val_preds = (val_preds_df["prob"].values > 0.5).astype(float)     # get the predicted labels
        val_true = [label for _, label in val_data]                     # get the true labels
        val_prob = val_preds_df["prob"].values                         # get the probability of the positive and negative class

        # print scores
        val_loss /= val_num_loss
        print(f"\tvalidation loss: {val_loss:.4f}")
        val_print_and_log_scores(fold, epoch, val_true, val_preds, val_prob)

        # log train and val loss
        loss_writer.writerow([fold, epoch, training_loss, val_loss])

        # check if the current epoch is most optimal based on the current minimum validation loss
        if val_loss < min_val_loss:
            # save model checkpoint to folder
            model_path = os.path.join(model_folder_path, model_name)
            torch.save(model.state_dict(), model_path)

            # update min validation loss value
            min_val_loss = val_loss

        # get and log the elapsed time for current epoch
        epoch_time = time.time() - start_time
        time_writer.writerow([fold, epoch, epoch_time])


    #############################################################################################################################################
    # testing
    #############################################################################################################################################
    print("TESTING")

    loaded_model = torch.load(model_path)

    test_preds_collector = []
    model.eval()

    # iterate through dataloader and run the model
    with torch.no_grad():
        for batch in tqdm(test_dataloader, total=len(test_dataloader)):
            batch["image"] = batch["image"].permute(0, 2, 3, 1)      

            images = batch['image'].to(device)
            labels = batch['label'].float().to(device)    
            image_ids = batch['image_id']

            logits = model.forward(images)
            
            pangolin_logits = logits[:, pangolin_indices].logsumexp(dim=1)
            pangolin_prob = torch.sigmoid(pangolin_logits)

            preds_df = pd.DataFrame(
                pangolin_prob.detach().cpu().numpy(),
                index=batch["image_id"],
                columns=["prob"]
            )
            test_preds_collector.append(preds_df)

    test_preds_df = pd.concat(test_preds_collector)
    test_preds = torch.tensor(test_preds_df["prob"].values > 0.5).long()        # get the predicted labels
    test_true = [label for _, label in test_data]                               # get the true labels
    test_prob = test_preds_df["prob"].values                                    # get the probability of the positive and negative class

    # print and log scores
    test_print_and_log_scores(fold, test_true, test_preds, test_prob)

    # error analysis on misclassified images
    plot_misclassified(fold, test_preds, test_data)

    # display confusion matrix
    fig, ax = plt.subplots(figsize=(10, 10))
    plt.title("confusion matrix - fold " + str(fold + 1))
    cm = ConfusionMatrixDisplay.from_predictions(
        test_true,
        test_preds,
        ax=ax,
        xticks_rotation=90,
        colorbar=True,
        normalize='true'
    )

    # flush writes to logging files
    train_csv.flush()
    time_csv.flush()
    test_csv.flush()
    loss_csv.flush()




fold 1 -------------------------------------------------------------------------------------------------

epoch 1
TRAINING


100%|██████████| 15/15 [02:49<00:00, 11.29s/it]


	training loss: 0.3744
	pangolin accuracy: 0.8230
	pangolin precision: 0.7462
	pangolin recall: 0.6599
	pangolin f1-score: 0.7004
	pangolin auc: 0.8913
	other accuracy: 0.8230
	other precision: 0.8525
	other recall: 0.8975
	other f1-score: 0.8744
	other auc: 0.1087
VALIDATION


100%|██████████| 2/2 [00:16<00:00,  8.27s/it]


	validation loss: 0.1709
	pangolin accuracy: 0.9623
	pangolin precision: 0.9412
	pangolin recall: 0.9412
	pangolin f1-score: 0.9412
	pangolin auc: 0.9984
	other accuracy: 0.9623
	other precision: 0.9722
	other recall: 0.9722
	other f1-score: 0.9722
	other auc: 0.0016

epoch 2
TRAINING


...

	training loss: 0.0008
	pangolin accuracy: 1.0000
	pangolin precision: 1.0000
	pangolin recall: 1.0000
	pangolin f1-score: 1.0000
	pangolin auc: 1.0000
	other accuracy: 1.0000
	other precision: 1.0000
	other recall: 1.0000
	other f1-score: 1.0000
	other auc: 0.0000
VALIDATION


100%|██████████| 2/2 [00:15<00:00,  7.99s/it]


	validation loss: 0.1343
	pangolin accuracy: 0.9434
	pangolin precision: 0.8889
	pangolin recall: 0.9412
	pangolin f1-score: 0.9143
	pangolin auc: 0.9967
	other accuracy: 0.9434
	other precision: 0.9714
	other recall: 0.9444
	other f1-score: 0.9577
	other auc: 0.0033
TESTING


100%|██████████| 5/5 [00:39<00:00,  8.00s/it]

	pangolin accuracy: 0.9538
	pangolin precision: 0.9268
	pangolin recall: 0.9268
	pangolin f1-score: 0.9268
	pangolin auc: 0.9956
	other accuracy: 0.9538
	other precision: 0.9663
	other recall: 0.9663
	other f1-score: 0.9663
	other auc: 0.0044
Misclassified Samples Found: 3
Misclassified Samples Found: 3


In [134]:
train_csv.flush()
val_csv.flush()
time_csv.flush()
test_csv.flush()
loss_csv.flush()

train_csv.close()
val_csv.close()
time_csv.close()
test_csv.close()
loss_csv.close()